In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob
import os
from PIL import Image
import itertools


import keras,os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.preprocessing.image import ImageDataGenerator


# example of tending the vgg16 model
from keras.applications.vgg16 import VGG16
from keras.models import Model
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

## EDA and Data Preprocessing

Before we can begin training/testing the model, EDA is required to assess what preprocessing will be needed.

In [8]:
meta_data = pd.read_csv("/content/drive/My Drive/CancerImages/HAM10000_metadata.csv")
meta_data.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear


In [9]:
meta_data['localization'].unique()

array(['scalp', 'ear', 'face', 'back', 'trunk', 'chest',
       'upper extremity', 'abdomen', 'unknown', 'lower extremity',
       'genital', 'neck', 'hand', 'foot', 'acral'], dtype=object)

In [15]:
lesion_type_dict = {
    'nv': 'Melanocytic nevi',
    'mel': 'Melanoma',
    'bkl': 'Benign keratosis-like lesions ',
    'bcc': 'Basal cell carcinoma',
    'akiec': 'Actinic keratoses',
    'vasc': 'Vascular lesions',
    'df': 'Dermatofibroma'
}

is_cancerous = {
    'nv': 0,
    'mel': 1,
    'bkl': 0,
    'bcc': 1,
    'akiec': 0,
    'vasc': 0,
    'df': 0
}

base_skin_dir = '/content/drive/My Drive/CancerImages'


imageid_path_dict = {os.path.splitext(os.path.basename(x))[0]: x
                     for x in glob(os.path.join(base_skin_dir, '*', '*.jpg'))}

In [16]:
# Add path to image to meta_data
meta_data['path'] = meta_data['image_id'].map(imageid_path_dict.get)
meta_data['is_cancerous'] = meta_data['dx'].map(is_cancerous.get)
# Add lesion type to image to meta_data
meta_data['cell_type'] = meta_data['dx'].map(lesion_type_dict.get)
# Convert this to categorical
meta_data['cell_type_idx'] = pd.Categorical(meta_data['cell_type']).codes

In [17]:
meta_data

,lesion_id,image_id,dx,dx_type,age,sex,localization,path,is_cancerous,cell_type,cell_type_idx
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,/content/drive/My Drive/CancerImages/HAM10000_...,0,Benign keratosis-like lesions,2
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,/content/drive/My Drive/CancerImages/HAM10000_...,0,Benign keratosis-like lesions,2
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,/content/drive/My Drive/CancerImages/HAM10000_...,0,Benign keratosis-like lesions,2
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,/content/drive/My Drive/CancerImages/HAM10000_...,0,Benign keratosis-like lesions,2
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,/content/drive/My Drive/CancerImages/HAM10000_...,0,Benign keratosis-like lesions,2
...,...,...,...,...,...,...,...,...,...,...,...
10010,HAM_0002867,ISIC_0033084,akiec,histo,40.0,male,abdomen,/content/drive/My Drive/CancerImages/HAM10000_...,0,Actinic keratoses,0
10011,HAM_0002867,ISIC_0033550,akiec,histo,40.0,male,abdomen,/content/drive/My Drive/CancerImages/HAM10000_...,0,Actinic keratoses,0
10012,HAM_0002867,ISIC_0033536,akiec,histo,40.0,male,abdomen,/content/drive/My Drive/CancerImages/HAM10000_...,0,Actinic keratoses,0
10013,HAM_0000239,ISIC_0032854,akiec,histo,80.0,male,face,/content/drive/My Drive/CancerImages/HAM10000_...,0,Actinic keratoses,0


## Setup for ML
Need to to add images to the dataset.

Then need to split into train/validation/test sets.


In [6]:
#Add images to meta_data
meta_data['image'] = meta_data['path'].map(lambda x: np.asarray(Image.open(x)))

df = meta_data.copy(deep=True)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-6-3399a5fa4a47>", line 2, in <cell line: 2>
    meta_data['image'] = meta_data['path'].map(lambda x: np.asarray(Image.open(x)))
  File "/usr/local/lib/python3.10/dist-packages/pandas/core/series.py", line 4539, in map
    new_values = self._map_values(arg, na_action=na_action)
  File "/usr/local/lib/python3.10/dist-packages/pandas/core/base.py", line 890, in _map_values
    new_values = map_f(values, mapper)
  File "pandas/_libs/lib.pyx", line 2924, in pandas._libs.lib.map_infer
  File "<ipython-input-6-3399a5fa4a47>", line 2, in <lambda>
    meta_data['image'] = meta_data['path'].map(lambda x: np.asarray(Image.open(x)))
  File "/usr/local/lib/python3.10/dist-packages/PIL/Image.py", line 3236, in open
    prefix = fp.read(16)
KeyboardInterrupt

During handling of the abov

TypeError: ignored

In [ ]:
# Select features of target
features = df.drop(columns=['cell_type_idx'], axis=1)
target = df['cell_type_idx']

# Split into test/train sets
x_train_o, x_test_o, y_train, y_test = train_test_split(features, target, test_size=0.20, random_state=1234)

# Select just images as x_train/x_test
x_train = np.asarray(x_train_o['image'].tolist())
x_test = np.asarray(x_test_o['image'].tolist())

x_train_mean = np.mean(x_train)
x_train_std = np.std(x_train)

x_train = (x_train - x_train_mean) / x_train_std
x_test = (x_test - x_train_mean) / x_train_std

y_train = to_categorical(y_train, num_classes = 7)
y_test = to_categorical(y_test, num_classes = 7)

# Split training into training and validation
x_train, x_validate, y_train, y_validate = train_test_split(x_train, y_train, test_size=0.1, random_state=2)

# Reshape image in 3 dimensions (height = 75px, width = 100px, channels = 3)
x_train = x_train.reshape(x_train.shape[0], *(75, 100, 3))
x_test = x_test.reshape(x_test.shape[0], *(75, 100, 3))
x_validate = x_validate.reshape(x_validate.shape[0], *(75, 100, 3))


## Model Setup

Now the data has been appropriately preprocessed, the model must be set up so it can be trained on the data.

In [ ]:
import tensorflow as tf
from keras.layers import Dropout, GlobalAveragePooling2D
input_shape = (75, 100, 3)
n_classes = len(list(lesion_type_dict.values()))

#Randomly flip, rotate and zoom the data
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal'),
  tf.keras.layers.RandomRotation(0.2),
  tf.keras.layers.RandomZoom(.5, .2)
])

#Rescale input to be [-1, 1] not [0, 255]
preprocess_input = tf.keras.applications.vgg16.preprocess_input

# Define structure of base model
base_model = tf.keras.applications.VGG16(input_shape=input_shape,
                                               include_top=False,
                                               weights='imagenet')

#Freeze the weights of base model
base_model.trainable = False


#Add a prediction layer
global_average_layer = GlobalAveragePooling2D()
prediction_layer  = Dense(n_classes, activation='softmax')


#Combine preprocessing and model
inputs = tf.keras.Input(shape=input_shape)
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = global_average_layer(x)
x = Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)
model.summary()

# Define the optimizer
base_learning_rate = 0.0001
optimizer = Adam(learning_rate= base_learning_rate)

# Compile the model

model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

In [ ]:
input_shape =
num_classes = 7


# load model without classifier layers
model = VGG16(include_top=False, input_shape=input_shape)

# mark loaded layers as not trainable
for layer in model.layers:
 layer.trainable = False


# add new classifier layers
flat1 = Flatten()(model.layers[-1].output)
class1 = Dense(64, activation='relu')(flat1)
output = Dense(num_classes, activation='softmax')(class1)
# define new model
model = Model(inputs=model.inputs, outputs=output)
# summarize
model.summary()
# ...

In [ ]:
from keras.utils import plot_model

# plot_model(model)

In [ ]:
# Train the model and get the history
history = model.fit(x_train, y_train, epochs=3, validation_data=(x_validate, y_validate))

# Plot training & validation loss values
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()


plt.show()

## Model Performance

In [ ]:
model.save("VGG_model.h5")

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Predict the values from the validation dataset
Y_pred = model.predict(x_test)

# # # Convert predictions classes to one hot vectors
Y_pred_classes = np.argmax(Y_pred,axis = 1)

# # Convert validation observations to one hot vectors
Y_true = np.argmax(y_test,axis = 1)

# Check the class names and their corresponding indices
class_names = list(lesion_type_dict.values())
# Get the class index mapping
code_to_name = {idx: class_name for idx, class_name in enumerate(class_names)}

# Display the confusion matrix with rotated x-axis labels
ConfusionMatrixDisplay.from_predictions(y_test, Y_pred_classes,display_labels=list(code_to_name.values()), cmap='inferno', normalize='true', values_format=".0%")
plt.xticks(rotation=90)
plt.grid(False)
plt.show()

In [20]:
# Check the class names and their corresponding indices
class_names = list(meta_data['cell_type_idx'].unique())
# Get the class index mapping
code_to_name = {idx: class_name for idx, class_name in zip(meta_data['cell_type_idx'].unique(), meta_data['cell_type'].unique())}
code_to_name

{2: 'Benign keratosis-like lesions ',
 4: 'Melanocytic nevi',
 3: 'Dermatofibroma',
 5: 'Melanoma',
 6: 'Vascular lesions',
 1: 'Basal cell carcinoma',
 0: 'Actinic keratoses'}